# Task 1
### Method (b) - Dense Static Representation

First we import all the libraries we want for this method, and import the training and test csv's.

In [99]:
from gensim.models import Word2Vec
import numpy as np

import pandas as pd
import plotly.graph_objects as go

import re
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from tqdm import tqdm

train_df = pd.read_csv('./Training-dataset.csv')
test_df = pd.read_csv('./Task-1-validation-dataset.csv', header=None)

Now let's clean up the training dataset's synopses to have no punctuation, tags, numbers or special characters.
This will get us an array of words.

In [101]:
cleaned_sentences = []
for synopsis in tqdm(list(train_df['plot_synopsis'])):
    sentences = re.split(r'\.|\?|\!', synopsis) # split the title/synopsis into sentences
    
    for sentence in sentences:
        # for sentence in sent_tokenize(synopsis):
        # Remove non-alphabetic characters and convert to lowercase
        sentence = re.sub('[^a-zA-Z]', ' ', sentence).lower()
        # Tokenise the sentence
        sentence = word_tokenize(sentence)
        # get a set of the stopwords to remove
        stop_words = set(stopwords.words('english'))
        # Remove stopwords
        sentence = [word for word in sentence if word not in stop_words]
        # Lemmatize the words
        lemmatizer = WordNetLemmatizer()
        sentence = [lemmatizer.lemmatize(word) for word in sentence]
    
        cleaned_sentences.append(sentence)

print("Number of sentences & titles to get context from: ", len(cleaned_sentences))

100%|██████████| 8257/8257 [02:36<00:00, 52.70it/s]

Number of sentences & titles to get context from:  394993


Call the model. This is where all the hyper parameters are set.

In [168]:
w2v = Word2Vec(cleaned_sentences, vector_size=200, window=10, min_count=1, workers=64)

Define a cosine similarity function.

This function takes in a model, and 2 words to find the cosine distance between.

We return 0 if the word is OOV (out of vocabulary).

In [169]:
def cosine_similarity(model, word_a, word_b):
    try:
        vector_a = model.wv[word_a]
    except:
        return 0
    try:
        vector_b = model.wv[word_b]
    except:
        return 0

    distance = np.dot(vector_a, vector_b) / (np.linalg.norm(vector_a) * np.linalg.norm(vector_b))
    
    return distance

print(cosine_similarity(w2v, 'bone', 'teeth'))


0.7855864


In [170]:
data = [] 
for index, row in test_df.iterrows():
    predicted_similarity = cosine_similarity(w2v, row[1], row[2])
    data.append([row[0], predicted_similarity])
    print(row[0], row[1], row[2], predicted_similarity)
output_df = pd.DataFrame(data)
print(output_df[0])

1 absorb learn 0.27871406
2 absorb withdraw 0.54757327
3 achieve accomplish 0.82440907
4 achieve try -0.00097129494
6 acquire get 0.20465411
7 acquire obtain 0.84708637
8 acquire find 0.1635732
11 apple sauce 0.708912
12 apple lemon 0.47581014
13 apple sunshine 0.5357237
14 area region 0.6556919
16 area corner 0.3516357
17 arm shoulder 0.7911532
20 arm body 0.38487807
21 arm vein 0.49134395
22 arm knee 0.71785605
23 arm bone 0.7021116
24 arm neck 0.7541852
25 ball cannon 0.4482337
26 ball basket 0.4906867
29 bath trick 0.33768323
30 bath wife 0.018425165
31 bath balloon 0.5620019
32 bed bedroom 0.77677757
33 bed blanket 0.6818809
34 bed crib 0.71027684
35 bed hospital 0.36410213
38 bed chair 0.75599235
39 belief opinion 0.544694
40 belief concept 0.46274665
41 belief impression 0.3484203
42 belief flower -0.08579012
43 bird hawk 0.5242088
44 bird turkey 0.49896047
45 bird hen 0.4276509
46 bird cock 0.5016093
48 bone jaw 0.68475544
49 bone ankle 0.6763656
50 bone knee 0.6837746
51 bone 

In [171]:
from pathlib import Path
filepath = Path('./10861383-Task1-method-b.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
print(output_df)
output_df.to_csv(filepath, index=False, header=False)  

       0         1
0      1  0.278714
1      2  0.547573
2      3  0.824409
3      4 -0.000971
4      6  0.204654
..   ...       ...
145  177  0.686436
146  178  0.423663
147  179  0.299940
148  181  0.355474
149  182  0.471883

[150 rows x 2 columns]
